이번 부록에서는 소프트맥스 함수와 교차 엔트로피 오차의 계산 그래프를 그려보고, 그 역전파를 구해보자. 소프트맥스 함수는 "Softmax"계층, 교차 엔트로피 오차는 "Cross Entropy Error"계층, 이 둘을 조합한 계층을 "Softmax_with_Loss"계층이라 한다. 결과를 먼저 보면 Softmax_with_Loss 계층은 다음과 같은 계산 그래프로 그릴 수 있다.

<img src=images/A_1.png height=100px width=500px>

이 계산 그래프에서는 3가지 클래스 분류를 수행하는 신경망을 가정하고 있다. 이전 계층으로부터의 입력은 $(a_1, a_2, a_3)$이며, Softmax 계층은 $(y_1, y_2, y_3)$를 출력한다. 또, 정답 레이블은 $(t_1, t_2, t_3)$이며 Cross_Entropy_Error 계층은 손실 L을 출력한다.

# A.1 순전파

위의 그림의 계산 그래프에서는 Softmax 계층과 Cross Entropy Error 계층의 내용은 생략하였다. 이번 절에서는 이 두 계층의 내용을 생략하지 않고 그리는 것부터 시작하자.<br>
우선은 Softmax 계층으로, 소프트맥스 함수는 수식으로 다음과 같다.

$
y_k = 
\frac{e^{a_k}}{\sum^{n}_{i=1} e^{a_k}}
\qquad$ [식 A.1]

그리고 아래 그림은 이 Softmax 계층의 계산 그래프이다.

<img src=images/A_2.png height=100px width=500px>

이 그림은 지수의 합, 즉 [식 A.1]의 분모 항을 S로 표기하였다. 또, 최종 출력은 (y_1, y_2, y_3)이다.

이어서 Cross Entropy Error 계층을 보자. 교차 엔트로피 오차의 수식은 다음과 같다.

$
L = 
-\sum_{k} t_k\log y_k
\qquad$ [식 A.2]

[식 A.2]를 바탕으로, Cross Entropy Error 계층의 계산 그래프는 다음과 같이 그릴 수 있다.

<img src=images/A_3.png height=100px width=500px>

이 그림은 [식 A.2]를 그대로 계산 그래프로 그린 것이다.

# A.2 역전파

우선 Cross Entropy Error 계층의 역전파이다. 이 계층의 역전파는 다음과 같이 그릴 수 있다.

<img src=images/A_4.png height=100px width=500px>

이 계산 그래프의 역전파를 구할 때는 다음을 유념해야 한다.
1. 역전파의 초깃값, 즉 위의 그림의 가장 오른쪽 역전파의 값은 1이다.(${\partial L \over \partial L} = 1$이므로)
2. "x"노드의 역전파는 순전파 시의 입력들의 값을 "서로 바꿔" 상류의 미분에 곱하고 하류로 흘린다.
3. "+"노드에서는 상류에서 전해지는 미분을 그대로 흘린다.
4. "log"노드의 역전파는 다음 식을 따른다. $ \\ y = \log x \\ {\partial y \over \partial x} = {1 \over x}$

이상의 규칙을 따르면 Cross Entropy Error 계층의 역전파는 쉽게 구할 수 있다. 결과는 
$
\begin{pmatrix}
-\frac{t_1}{y_1}  & -\frac{t_2}{y_2} & -\frac{t_3}{y_3}
\end{pmatrix}
$이며, 이 값이 Softmax 계층으로의 역전파 입력이 된다.

이어서 Softmax 계층의 역전파이다. 이 계층의 역전파는 조금 복잡하니 하나씩 확인하면서 진행해보자.

#### 1단계
<img src=images/A_5_1.png height=100px width=500px>

앞 계층의 역전파 값이 흘러온다.

#### 2단계
<img src=images/A_5_2.png height=100px width=500px>

곱셈의 역전파 규칙에 따라 "/"노드로 가는 역전파에는 각각 $e^{a_1}, e^{a_2}, e^{a_3}$를 곱해준다. 이후, $y_k = \frac{e^{a_k}}{S}$이므로 역전파 값은 $-t_kS$가 된다.<br>
수식으로 정리하면 다음과 같다.

$
-\frac{t_1}{y_1}e^{a_1} = 
-t_1\frac{S}{e^{a_1}}e^{a_1} = 
-t_1S
\qquad$ [식 A.3]

#### 3단계
<img src=images/A_5_3.png height=100px width=500px>

"/"노드에서는 우선은 S 하나로 부터 3개의 노드의 변화를 야기하므로, 역전파의 입력들이 모여야 한다. 따라서 각 역전파들을 더해주면 된다. 각각의 역전파는 다음과 같은 규칙에 따른다.

$
if\quad y = {1 \over x} \qquad 
then\quad {\partial y \over \partial x} = 
-\frac{1}{x^2}
\qquad$

따라서 역전파는 다음과 같이 된다. $\frac{1}{S}(t_1 + t_2 + t_3)$이 된다. 그런데 여기에서 (t_1, t_2, t_3)은 "원-핫-벡터"로 표현된 정답 레이블이므로 오직 정답만이 1을 가리키고 나머지는 0이므로 $t_1 + t_2 + t_3 = 1$이 된다. 따라서 최종적인 역전파는 $\frac{1}{S}$가 된다.

#### 4단계
<img src=images/A_5_4.png height=100px width=500px>

"+"노드는 입력을 여과 없이 내보낼 뿐이다.

#### 5단계
<img src=images/A_5_5.png height=100px width=500px>

"x"노드이므로 역전파를 계산해주면 다음과 같다.

$
-\frac{t_1}{y_1}\frac{1}{S} = 
-\frac{t_1}{e^{a_1}}
\qquad$

여기서도 $y_1 = \frac{e^{a_1}}{S}$를 이용하였다.

#### 6단계
<img src=images/A_5_6.png height=100px width=500px>

"exp"노드에서는 다음 관계식이 성립된다.

$
y = e^{x} \\
{\partial y \over \partial x} = e^{x}
\qquad$ [식 A.4]

그리고 두 갈래의 입력의 합에 $e^{a_1}$를 곱한 수치가 여기에서 구하는 역전파이다. 식으로 쓰면 
$
(\frac{1}{S} - \frac{t_1}{e^{a_1}})e^{a_1}
$
이 되고, 이를 변형하면 $y_1 - t_1$이 된다. 이상에서 순전파의 입력이 $a_1$인 첫 번째 노드에서는 역전파가 $y_1 - t_1$임이 유도되었다. 나머지 $a_2$와 $a_3$의 역전파도 같은 순서로 구할 수 있다. 결과는 각각 $y_2 - t_2$와 $y_3 - t_3$가 된다. 또, 여기에서 다룬 3가지 클래스 분류 외에, 가령 n클래스 분류에서도 같은 결과가 유도되는 것은 쉽게 알아낼 수 있다.